In [2]:
import tensorflow as tf
import numpy as np
from keras.models import Sequential
import cv2
import skimage
import random
import os
from sklearn.neighbors import NearestNeighbors
from imgaug.imgaug import augmenters as iaa
from densenet121 import DenseNet
from sklearn import decomposition
from sklearn.neighbors import KNeighborsClassifier
from keras.layers import Dense
from keras.models import Model
from keras.optimizers import SGD

Using TensorFlow backend.


In [3]:
class image_util:
    def __init__(self, data_dir, biz_label_file_name, photo_biz_file_name):
        self.batch_index = 0
        self.image_paths = [os.path.join(data_dir,i) for i in os.listdir(data_dir) if i.endswith('.jpg') and not i.startswith("._")]
        random.shuffle(self.image_paths)
        self.images = []
        self.labels = []
        self.image_len = len(self.image_paths)
        self.one_hot = self.read_csv_one_hot(biz_label_file_name)
        self.photo_biz = self.photo_to_biz_id(photo_biz_file_name)

    def next_batch(self, batch_size):
        images = []
        labels = []
        if batch_size + self.batch_index * batch_size < self.image_len:
            start = self.batch_index
            end = batch_size + self.batch_index
            for path in self.image_paths[start:end]:
                img = cv2.imread(path)
                photo_id = os.path.basename(path).split(".")[0]
                self.labels.append(np.zeros(9))
                img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
                images.append(img)
                labels.append(np.zeros(9))
            images = np.asarray(images)
            labels = np.asarray(labels)
            self.batch_index += 1
            return images, labels, 1
        else:
            self.batch_index = 0
            start = self.batch_index
            end = batch_size + self.batch_index
            for path in self.image_paths[start:end]:
                img = cv2.imread(path)
                photo_id = os.path.basename(path).split(".")[0]
                self.labels.append(np.zeros(9))
                img = cv2.resize(img,(224,224),interpolation = cv2.INTER_AREA)
                images.append(img)
                labels.append(np.zeros(9))
            images = np.asarray(images)
            labels = np.asarray(labels)
            return images, labels, 0
    
    def read_csv_one_hot(self, file_name):
        with open(file_name,"r") as f:
            lines = f.readlines()[1:]
        biz_id_to_label = {}
        for line in lines:
            try:
                biz_id_to_label[line.split(",")[0]] = np.zeros(9)
                for label in line.split(",")[1].rstrip().split(' '):
                    biz_id_to_label[line.split(",")[0]][int(label)]=1
            except:
                if not line.split(",")[1].rstrip():
                    continue
        return biz_id_to_label
    
    def photo_to_biz_id(self, file_name):
        with open(file_name,"r") as f:
            lines = f.readlines()[1:]
        photo_to_biz = {}
        for line in lines:
            photo_to_biz[line.split(",")[0]] = line.split(",")[1].rstrip() 
        return photo_to_biz
        

In [4]:
util = image_util('/home/rendaxuan/Documents/workspace/4032/test_photos', '/home/rendaxuan/Documents/workspace/4032/train.csv', '/home/rendaxuan/Documents/workspace/4032/test_photo_to_biz.csv')

In [5]:
model = DenseNet(reduction=0.5, classes=1000, weights_path='./densenet121_weights_tf.h5')
print(model.layers[-1].output_shape)
model.layers.pop()
model.layers.pop()
print(model.layers[-1].output_shape)
output = model.layers[-1].output
new_model = Model(model.input, output)
sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

densenet121.py:45: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (7, 7), strides=(2, 2), use_bias=False, name="conv1")`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
densenet121.py:98: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), use_bias=False, name="conv2_1_x1")`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', bias=False)(x)
densenet121.py:108: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), use_bias=False, name="conv2_1_x2")`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base+'_x2', bias=False)(x)
densenet121.py:164: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  concat_feat = merge([concat_feat, x], mode='concat', concat_axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))
/usr/local/lib/python2.7/dist-packa

(None, 1000)
(None, 1024)


In [7]:
batch_size = 1000
images_features = []
images_labels = []
images, labels, go_on = util.next_batch(batch_size)
counter = 0
import time
start = time.time()
while go_on:
    images_features.append(new_model.predict(images))
    images_labels.append(labels)
    images, labels, go_on = util.next_batch(batch_size)
    print("==================================================")
    print(str(counter*1000/238400.0*100)+"%   "+str(time.time()-start))
    start = time.time()
    counter += 1
features_test = np.vstack(images_features)
features_labels_test = np.vstack(images_labels)

0.0%   10.212141037
0.419463087248%   10.2485749722
0.838926174497%   10.2539219856
1.25838926174%   10.2725760937
1.67785234899%   10.3976368904
2.09731543624%   10.3929901123
2.51677852349%   10.379873991
2.93624161074%   10.3718020916
3.35570469799%   10.3952400684
3.77516778523%   10.4013171196
4.19463087248%   10.3144910336
4.61409395973%   10.3198070526
5.03355704698%   10.3195989132
5.45302013423%   10.4338419437
5.87248322148%   10.297068119
6.29194630872%   10.2906508446
6.71140939597%   10.3008830547
7.13087248322%   10.2701361179
7.55033557047%   10.3011729717
7.96979865772%   10.3063468933
8.38926174497%   10.3102748394
8.80872483221%   10.3182919025
9.22818791946%   10.3105390072
9.64765100671%   10.3003211021
10.067114094%   10.3160820007
10.4865771812%   10.3883109093
10.9060402685%   10.3214988708
11.3255033557%   10.2986690998
11.744966443%   10.2884590626
12.1644295302%   10.2963278294
12.5838926174%   10.3037209511
13.0033557047%   10.3198161125
13.4228187919%   10.3

In [12]:
features_test.shape

(234000, 1024)

In [10]:
np.save('/home/rendaxuan/Documents/workspace/4032/features_test.npy', features_test)

In [11]:
np.save('/home/rendaxuan/Documents/workspace/4032/features_labels_test.npy', features_labels_test)

In [3]:
features = np.load('/home/rendaxuan/Documents/workspace/4032/features_234000.npy')

In [7]:
features_labels = np.load('/home/rendaxuan/Documents/workspace/4032/features_labels_234000.npy')

In [61]:
features_train = features[:200000]

In [62]:
features_train_labels = features_labels[:200000]

In [63]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

n_neighbors = 3
clf = neighbors.KNeighborsClassifier(n_neighbors)
clf.fit(features_train, features_train_labels)
# Z = clf.predict(features[233000])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [ ]:
# Z = clf.predict(np.expand_dims(features[233000],axis=0))

In [64]:
Z = clf.predict(features[-500:])

In [65]:
Z.shape

(500, 9)

In [77]:
clf.score(np.random.random((2000,1024)),features_labels[-2000:])

0.037499999999999999

In [44]:
features[-1000:-4000:-1].shape

(3000, 1024)

In [45]:
features_labels[-1000:-4000:-1].shape

(3000, 9)

In [60]:
from sklearn.utils import shuffle
features, features_labels = shuffle(features, features_labels, random_state=0)

In [72]:
a = np.random.random((10,1024))

In [74]:
a.shape


(10, 1024)